### Aikasarjaharjoitustyö datojen poiminta ja yhdistäminen.  
Tässä tiedostossa Käsitellään ilmatieteenlaitoksen data, niin, että sitä voidaan käyttää lineaarisen regression tai muiden algoritmien kanssa.

### Tuodaan datan käsittelyssä ja kuvantamisessa tarvittavat kirjastot. 

In [13]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Säädatan lataus, sarakkeiden poiminta ja datan tarkastelu sekä käsittely

Ladataan tiedot dataframeen. Käytetään materiaalina toisessa yhteydessä hyödynnettyä viinidataa. Muunnetaan ilmatieteenlaitoksen tarjoama aikamuoto formaattiin vvvv-kk-pp tt:mm. Dropataan turhat sarakkeet ja vaihdetaan sarakejärjestys.

In [14]:
df = pd.read_excel('Temp_data.xlsx')
df = df.rename(columns={'Keskituulen nopeus [m/s]':'wind','Lämpötilan keskiarvo [°C]':'C','Aika [Paikallinen aika]':'Aika','[W/m2]':'Wperm2'}) #Muunnetaan sarakkeiden nimet

df['DS'] = df['Vuosi'].astype(str) + '-' + df['Kuukausi'].astype(str).str.zfill(2) + '-' + df['Päivä'].astype(str).str.zfill(2) + ' ' + df['Aika'] #Tehdään aikamuunnos
df=df.drop(['Havaintoasema','Vuosi','Kuukausi','Päivä','Aika','Ylin lämpötila [°C]','Alin lämpötila [°C]'], axis=1) #Dropataan turhat sarakkeet

df_C = df.loc[:, ['DS', 'C','wind','hPa','moist%','Wperm2']] # Vaihdetaan sarakejärjestys
print(df_C.head())

                 DS     C wind     hPa moist% Wperm2
0  2019-05-23 00:00  16.4  2.5  1007.1     82   -4.8
1  2019-05-23 01:00  15.6  2.1  1007.1     83   -5.2
2  2019-05-23 02:00  14.8  3.4  1007.6     90   -4.8
3  2019-05-23 03:00  14.8  4.5  1007.8     96   -3.7
4  2019-05-23 04:00  14.8  4.9    1008     97   -2.9


Katsotaan sarakkeiden datatyypit.

In [15]:
print(df_C.dtypes)

DS        object
C         object
wind      object
hPa       object
moist%    object
Wperm2    object
dtype: object


Tehdään datatyyppien muunnokset.

In [16]:
df_C['DS'] = pd.to_datetime(df_C['DS'])
df_C['DS'] = df_C['DS'].dt.strftime('%Y-%m-%d %H:%M')
print(df_C.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41104 entries, 0 to 41103
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DS      41104 non-null  object
 1   C       41104 non-null  object
 2   wind    41104 non-null  object
 3   hPa     41089 non-null  object
 4   moist%  41089 non-null  object
 5   Wperm2  41089 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB
None


Edellä olevat voidaan myös todentaa tarkastelemalla muutujien tilastollisia tietoja. Jokaisella muuttujalla on variaatiota. (std <> 0 )

In [17]:
df_C.describe()

,DS,C,wind,hPa,moist%,Wperm2
count,41104,41104.0,41104.0,41089.0,41089,41089.0
unique,41099,573.0,141.0,779.0,85,6506.0
top,2020-10-25 03:00,0.3,3.5,1016.5,100,-0.5
freq,2,279.0,841.0,202.0,3060,1090.0


Datassa sarakkeessa DS näyttää olevan duplikaatteja huolimatta siitä, että näin ei pitäisi olla. Tarkastetaan, mistä on kyse. Katsotaan mitä ne ovat.

In [18]:
duplicate_values = df_C[df_C.duplicated(subset=['DS'], keep=False)]

print("Duplicate values in column 'DS':")
print(duplicate_values)

Duplicate values in column 'DS':
                     DS    C wind     hPa moist% Wperm2
3765   2019-10-27 03:00  3.6  3.9  1001.1    100   -1.6
3766   2019-10-27 03:00  2.9  3.4  1001.1    100   -1.6
12485  2020-10-25 03:00    3  3.1  1005.9     96     -1
12486  2020-10-25 03:00    3  2.8  1005.9     96     -1
21362  2021-10-31 03:00  8.8  2.8  1013.2     87   -0.4
21363  2021-10-31 03:00  8.8  2.4  1013.2     87   -0.4
30097  2022-10-30 03:00  5.5  7.7  1003.6     67   -3.1
30098  2022-10-30 03:00  5.3  6.7  1003.6     67   -3.1
38827  2023-10-29 03:00 -1.8  2.3  1007.2     67   -0.5
38828  2023-10-29 03:00 -1.8  2.5  1007.2     67   -0.5


Duplikaatit näyttävät aiheutuvan kellojen siirrosta. Lienee järkevää poistaa niistä toinen. 

In [19]:
df_C.drop_duplicates(subset=['DS'], keep='first', inplace=True)

Lämpötiladata näyttää olevan on OK. 

In [23]:
df_C.to_excel('C.xlsx', index=False) #Viedään käsitelty data tiedostoon, Kokeilumielessä haluan käyttää xlsx tiedostoja, vaikka niiden käsittely on tehottomampaa.